## Import library

In [1]:
import numpy as np
import pandas as pd
from chineseTextLib import *
from chineseDict import *
import os
import json
from pyvis.network import Network
from helper import timer

## Create character & word dataframe

In [2]:
fileName="zfl"
baseInputLink = f"./zfl/wordCountInput/{fileName}.txt"
dfWords=createWordListFromText(baseInputLink)
dfKnownWords=createCharacterListFromText(baseInputLink)
# dfknownWords
dfWords[:3]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Admin\AppData\Local\Temp\jieba.cache
Loading model cost 0.682 seconds.
Prefix dict has been built successfully.


,traditionalChinese,times
0,我們,880
1,這個,766
2,修煉,686


## Slice word list for scraping data online

In [3]:
#create conent column with NaN value
dfWords2Process=dfWords
dfWords2Process['content']=np.nan
print(dfWords2Process.head(1))

#create slice of charSet then save to picle file type
list=np.arange(0,len(dfWords),100)
list=np.append(list,len(dfWords))
# print(list)
i=0
for i in range(len(list)-1):
    if os.path.exists(f"./data/charSet/charSet {i:02d}-{list[i]}-{list[i+1]}.pkl"):
        pass
        # print("Already slice. Go ahead")
    else:
        dfWords2ProcessSlice=dfWords2Process[list[i]:list[i+1]]
        dfWords2ProcessSlice.to_pickle(f"./data/charSet/charSet {i:02d}-{list[i]}-{list[i+1]}.pkl")

  traditionalChinese  times  content
0                 我們    880      NaN


## Get data for each slide then save to pickle

In [4]:
# #get phrase content for each slices of charSet
# currentFolderPath = os.getcwd()
# folder2Process=f"{currentFolderPath}/data/charSet"
# for root, directories, files in os.walk(folder2Process):
#     #loop through files
#     for f in files:
#         if os.path.exists(f"{folder2Process}/output/{f}")==True:
#             pass
#             print(f"already existed: {folder2Process}/ouput/{f}")
#         else:
#             print("Start craw new datas")
#             dfTemp2PrcessCharSet=pd.read_pickle(f"{folder2Process}/{f}")
#             print(f"Processing {folder2Process}/{f}")
#             #loop through dataFrame rows
#             for index,row in dfTemp2PrcessCharSet.iterrows(): 
#                 #check to get phrase content if null
#                 if pd.isnull(row['content']):
#                     # print(row['traditionalChinese'])
#                     content=getPhraseContent(row['traditionalChinese'])
#                     dfTemp2PrcessCharSet.at[index,'content']=content
#             dfTemp2PrcessCharSet.to_pickle(f"{folder2Process}/output/{f}")

## Create word detail column for node lookup

In [5]:
dfallCharSet=pd.read_pickle("./data/charSet/output/allCharSet.pkl")
dfallCharSet=dfallCharSet[dfallCharSet.chineseDetails!=""]
dfallCharSet['wordDetails']=dfallCharSet.apply(lambda x:f"""
{x.pinyin} | {x.zhuyin}
{x.traChinese} | {x.simChinese} -- {x.times}
{x.vietnameseDetails}
{x.chineseDetails}
{x.englishDetails}""", axis = 1)
dfallCharSet[:1]

,times,percentile,traChinese,simChinese,pinyin,zhuyin,chineseDetails,englishDetails,vietnameseDetails,wordDetails
0,880,100.0,我們,我们,wǒ men,ㄨㄛˇ ㄇㄣ,(1).稱包括本身在內的若干人。《紅樓夢·第四三回》：「我們不敢和老太太並肩，自然矮一等。」...,"---(E)we, us, ourselves, our",---(V)1. chúng tôi; chúng ta; chúng tao; chúng...,\nwǒ men | ㄨㄛˇ ㄇㄣ\n我們 | 我们 -- 880\n---(V)1. ch...


## Prepare Unihan dict data for lookup

In [6]:
unihanDict=convertTxtData2Dict(txtObject="dictionary.txt")
print(len(unihanDict))

#convert to list of dict
unihanDictList=[]
for value in unihanDict.values():
    jsonValue=json.loads(value)
    unihanDictList.append(jsonValue)

indexRange=np.arange(0,len(unihanDict))
dfUnihanDict=pd.DataFrame(unihanDictList,index=indexRange)
#Convert etymology dict-column to columns; then remove it and matches
dfUnihanDict=(dfUnihanDict
                    .join(pd.json_normalize(dfUnihanDict.pop('etymology')))
                    .drop('matches',axis=1)
                )

saveDict2JSON(unihanDictList,"hanziDictList.json")
dfUnihanDict[:1]

9574


,character,definition,pinyin,decomposition,radical,type,hint,phonetic,semantic
0,⺀,ice,[],？,⺀,NaN,NaN,NaN,NaN


## Merge Character df and dfUnihanDict

In [7]:
dfCharacter=dfKnownWords.merge(dfUnihanDict,left_on="word",right_on="character")
dfCharacter=dfCharacter.drop(['word'],axis=1)
dfCharacter[:1]
dfCharacter['percentile']=np.flip(np.arange(1,len(dfCharacter)+1)/len(dfCharacter)*100)

dictCharacter=dfCharacter.to_dict()
print(len(dictCharacter['character']))
dfCharacter[:1]

2035


,times,character,definition,pinyin,decomposition,radical,type,hint,phonetic,semantic,percentile
0,7236,的,"aim, goal; of; possessive particle; -self suffix",[de],⿰白勺,白,NaN,NaN,NaN,NaN,100.0


## Create character detail column for node lookup and save to characterDataZFL.pkl

In [8]:
if os.path.exists("characterDataZFL.pkl"): # Or folder, will return true or false:
    print("File created. Go ahead!")
    dfCharacter=pd.read_pickle('characterDataZFL.pkl')
else:
    dataFrameColumn=dfallCharSet['traChinese']
    dfCharacter['words'] = dfCharacter['character'].map(lambda x: relatedWordsZFL(x,dataFrameColumn))
    
    #combine character details
    dfCharacter['characterDetails']=dfCharacter.apply(lambda x:f"""
    {x.pinyin} | {x.decomposition} -- {x.times}
    {x.type} | sound:{x.phonetic} -- mean: {x.semantic}
    hint -- {x.hint}
    meaning -- {x.definition}
    words -- {x.words}""", axis = 1)
    
    # Save to pickle
    dfCharacter.to_pickle('characterDataZFL.pkl')
dfCharacter[:1]

File created. Go ahead!


,times,character,definition,pinyin,decomposition,radical,type,hint,phonetic,semantic,percentile,words,characterDetails
0,7236,的,"aim, goal; of; possessive particle; -self suffix",[de],⿰白勺,白,NaN,NaN,NaN,NaN,100.0,"[目的, 似的, 有的是, 重重的]",\n ['de'] | ⿰白勺 -- 7236\n nan | sound:na...


## Set network options

In [9]:
options="""
const options = {
  "nodes": {
    "borderWidth": 0,
    "borderWidthSelected": 5,
    "opacity": 0.8,
    "labelHighlightBold": false,
    "shadow": {
      "enabled": true
    },
    "shape": "circle",
    "title":{
      "multi": "html",
      "color":"white",
      "size":24,
      "font": {
        "size": 24
    }
    },
    "size": null,
    "font": {
      "size": 24
    }
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "labelHighlightBold": false,
    "selfReferenceSize": null,
    "selfReference": {
      "angle": 0.7853981633974483
    },
    "smooth": {
      "type": "diagonalCross",
      "forceDirection": "none"
    },
    "width":0
  },
  "interaction": {
    "navigationButtons": true,
    "tooltipDelay": 50,
    "zoomSpeed": 5
  },
  "physics": {
    "repulsion": {
      "nodeDistance": 300,
      "central_gravity":0.8,
      "springLength": 50,
      "spring_strength":0.05,
      "damping":0.08
    },
    "maxVelocity": 50,
    "minVelocity": 5,
    "solver": "repulsion"
  }
}
"""

## Prepare final dictCharacter for Network

In [10]:
dictCharacter=pd.read_pickle("characterDataZFL.pkl")

@timer
def updateWordDetails(listOfWords,df):
    tempWordDetailList=[]
    for i in range(len(listOfWords)):
        wordDetail=getWordContent(listOfWords[i],df)
        tempWordDetailList.append(wordDetail)
    return tempWordDetailList

if os.path.exists("dfCharacterZFL.pkl"):
    dictCharacter=pd.read_pickle("dfCharacterZFL.pkl")
    pass
else:
    dictCharacter['wordDetails']=dictCharacter['words'].map(lambda x: updateWordDetails(x,dfallCharSet))
    dictCharacter.to_pickle("dfCharacterZFL.pkl")
dictCharacter[:1]

,times,character,definition,pinyin,decomposition,radical,type,hint,phonetic,semantic,percentile,words,characterDetails,wordDetails
0,7236,的,"aim, goal; of; possessive particle; -self suffix",[de],⿰白勺,白,NaN,NaN,NaN,NaN,100.0,"[目的, 似的, 有的是, 重重的]",\n ['de'] | ⿰白勺 -- 7236\n nan | sound:na...,[\nmù dì | ㄇㄨˋ ㄉㄧˋ\n目的 | 目的 -- 17\n---(V)1. mụ...


## Generate NetWork

In [18]:
def map_data(characterData,shape='circle',radColor="#ece6d0",bgColor="#222222"):
    g=Network(notebook=True,height='900px',width="100%",bgcolor=bgColor,font_color="black",
              select_menu=False,neighborhood_highlight=True,filter_menu=True,cdn_resources="in_line")
    for node in characterData:
        if node['times']==1:
            node['color']="#ffffff"
            node['group']=4
        else:
            result=getColorBasedOnPercentile(node['percentile'])
            node['color']=result[0]
            node['group']=result[1]
    order=1
    for list in listOfRadList:
        # print(list)
        for node in characterData:
            if node['radical'] in list:
                g.add_node(node['character'],shape=shape,color=node['color'],title=node['characterDetails'])
                g.add_node(node['radical'],shape=shape,color=radColor)
                g.add_edge(node['radical'],node['character'],color=node['color'])
                for i in range(len(node['words'])):
                    g.add_node(node['words'][i],shape="box",color="white",title=node['wordDetails'][i])
                    g.add_edge(node['character'],node['words'][i],color=node['color'])
        # g.repulsion(node_distance=80,central_gravity=0.2,spring_length=50,spring_strength=0.05,damping=0.09)
        # g.set_edge_smooth('diagonalCross')
        # g.show_buttons(True)
        g.set_options(options)
        g.prep_notebook()
        g.show(f"N{order}--{list}.html",local=True)
        #reset network
        g=Network(notebook=True,height='900px',width="100%",bgcolor=bgColor,font_color='black',
              select_menu=False,neighborhood_highlight=True,filter_menu=False,cdn_resources="in_line")
        order+=1

#Slice list of radical to make each html file get x nodes
# listOfRadList=sliceRadicalListFromDf(dfKradical,np.ceil(len(dfKradical)/2))
listOfRadList=sliceRadicalListFromDf(dfCharacter,150)

dictCharacter=pd.read_pickle("dfCharacterZFL.pkl")
# dfTest=pd.DataFrame(dictCharacter[:300]['words']).to_pickle("testData.pkl")
columns2keep=['character','percentile','words','characterDetails','wordDetails','times','radical']
dictCharacter=dictCharacter[columns2keep]
dictCharacter=dictCharacter.to_dict('records')
# dictCharacter[:10]
map_data(dictCharacter)

<class 'pandas.core.series.Series'>
['扌', '口', '氵', '亻', '木言', '辶⺼土', '艹糹心女', '阝宀日釒忄', '石刂目⺮火足車', '貝禾彳頁广王攵力', '田疒大犭門一囗穴米儿巾', '虫耳酉寸止礻十隹子飠雨戈手欠糸']
